In [9]:
import sys
sys.path.append("..")
from src.Eval import compute_results_jsonl, compute_results_txt
from src.Utils import get_true_pred_entities, read_IOB_file
from baseline.parse_output import get_results_overall_table, get_results_table, parse_preds_baseline

BASELINES = ["bert-large-uncased", "roberta-large", "mpnet-base"]
OUTPUT_PATH_REDDIT = "../baseline/music-ner-eacl2023/output/dataset"


# Overall
## Baseline: BERT etc.

In [12]:
for i in range(1,5):
    print(get_results_overall_table(BASELINES, OUTPUT_PATH_REDDIT + str(i)).loc["macro", "strict", "f1"])


Model
bert-large-uncased    0.780469
mpnet-base            0.768840
roberta-large         0.753986
Name: (macro, strict, f1), dtype: float64
Model
bert-large-uncased    0.806217
mpnet-base            0.829565
roberta-large         0.782475
Name: (macro, strict, f1), dtype: float64
Model
bert-large-uncased    0.772230
mpnet-base            0.770317
roberta-large         0.759375
Name: (macro, strict, f1), dtype: float64
Model
bert-large-uncased    0.692649
mpnet-base            0.704873
roberta-large         0.715011
Name: (macro, strict, f1), dtype: float64


## LLaMa-3

In [19]:
for i in range(1,5):
    try:
        print(f"\nDataset{i}: ")
        data_llm_zeroshot = compute_results_jsonl(
            f"../baseline/music-ner-eacl2023/data/dataset{i}/test.bio", 
            f"../output/reddit/llama3_dataset{i}_simple2_4shot.jsonl")
    except:
        print(f"Dataset{i} Does not work")


Dataset1: 
Dataset1 Does not work

Dataset2: 


100%|██████████| 600/600 [00:00<00:00, 9523.45it/s]


Retag IOBs...


100%|██████████| 600/600 [00:00<00:00, 26315.00it/s]
2024-06-21 07:58:49 root INFO: Imported 599 predictions for 599 true examples


INFO: 1 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.702461  0.620553  0.658972        314           42          0       150          91         506       447
ent_type     0.751678  0.664032  0.705142        336           20          0       150          91         506       447
exact        0.747204  0.660079  0.700944        334           22          0       150          91         506       447

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.760504  0.635088  0.692161        181           12          0        92          45 

...

INFO: 3 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.68623   0.586873  0.632674        304           40          0       174          99         518       443
ent_type     0.75395   0.644788  0.695109        334           10          0       174          99         518       443
exact        0.708804  0.606178  0.653486        314           30          0       174          99         518       443

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.73251   0.607509  0.664179        178           10          0       105          55 

100%|██████████| 751/751 [00:00<00:00, 9855.46it/s]


Retag IOBs...


100%|██████████| 751/751 [00:00<00:00, 27525.86it/s]
2024-06-21 07:58:50 root INFO: Imported 750 predictions for 750 true examples


INFO: 1 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.690299  0.568356  0.62342         370           62          0       219         104         651       536
ent_type     0.755597  0.62212   0.682393        405           27          0       219         104         651       536
exact        0.740672  0.609831  0.668913        397           35          0       219         104         651       536

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.743243  0.575916  0.648968        220           24          0       138          52 

In [2]:
a,b = read_IOB_file("../baseline/music-ner-eacl2023/data/dataset1/test.bio")


# WHY IS MORE PREDICTED?


100%|██████████| 600/600 [00:00<00:00, 9161.34it/s]


Retag IOBs...


100%|██████████| 600/600 [00:00<00:00, 23768.03it/s]
2024-06-21 07:09:14 root INFO: Imported 599 predictions for 599 true examples


INFO: 1 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.702461  0.620553  0.658972        314           42          0       150          91         506       447
ent_type     0.751678  0.664032  0.705142        336           20          0       150          91         506       447
exact        0.747204  0.660079  0.700944        334           22          0       150          91         506       447

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.760504  0.635088  0.692161        181           12          0        92          45 

### Takeaways from Overall
- at least for few-shot, BERT-like models seem to be better than LLaMa-3

# Error Analysis

In [33]:
data = get_true_pred_entities(
    "bert-large-uncased", "../baseline/music-ner-eacl2023/output/dataset" + str(4)).dropna(
        how="all")

mask_artist = (data.Artist_true != data.Artist_pred) & ~(data.Artist_true.isna() & data.Artist_pred.isna())
mask_woa = (data.WoA_true != data.WoA_pred) & ~(data.WoA_true.isna() & data.WoA_pred.isna())


## Errors at WoA

In [35]:
data.loc[mask_woa, ["WoA_true", "WoA_pred"]]


,WoA_true,WoA_pred
6,[melody noir],[melody]
18,[molly],[lil dicky]
19,[the times they are a changin],[they are a changin]
44,[asesina],[sowlo]
54,[black licorice],NaN
...,...,...
716,[harvest moon],"[poolside, harvest moon]"
719,"[runnin, the creed 2]","[runnin, creed 2]"
720,[searching],NaN
730,"[vincent, imagine]",NaN


## Errors at Artist

In [36]:
data.loc[mask_artist, ["Artist_true", "Artist_pred"]]


,Artist_true,Artist_pred
6,[patrick watson],"[noir, patrick watson]"
11,NaN,[deathstep]
18,"[brandon urie, lil dicky]","[molly, brandon urie]"
19,NaN,[the times]
31,[elliot smith],"[grundge, elliot smith]"
...,...,...
716,"[poolside, neil young]",[neil young]
720,[youth 83],[searching youth]
730,NaN,"[vincent, imagine]"
743,[joshua speedy],NaN


# Ideas
### Stabilize
- Always: "Does the entity X refer to a Work of Art or a musical Artist?"
- If no entity found: "Is there no entity referring to a Work of Art (WoA) or a musical Artist?"
### Understandability
- If there is a WoA in this text, how did you identify it?
- If there is an Artist in this text, how did you identify it?
- Did you use linguistic knowledge to identify entities or did you 
